In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import math
import numpy as np
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, features
from folium.plugins import HeatMap, MarkerCluster
from src.functions import *

In [2]:
df = spain_offices()
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,,39.640730,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,NaN,NaN
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
...,...,...,...,...,...,...
161,FRV,Madrid,1,"C/ Fortuny, 45 bajo izquierda",NaN,NaN
162,FRV,Madrid,1,Rafael Calvo 42,NaN,NaN
163,StreamUK,Madrid,1,c/marathon 6,NaN,NaN
164,AdLINK Group,Madrid,1,affilinet Espana SLU,NaN,NaN


In [ ]:
df['address'].replace('', np.nan, inplace=True)

In [ ]:
df = df.dropna(subset=["address", "latitude", "longitude"], how='all')

In [ ]:
def get_coordinates (place):
    try:
        res = requests.get(f"https://geocode.xyz/{place}?json=1").json()
        return [res["longt"],res["latt"]]
    except:
        return [np.nan, np.nan]

In [ ]:
for i in df.index.values.tolist():
    if math.isnan(df.at[i, "latitude"]) or df.at[i, "latitude"]==0:
        address_complete = df.at[i, "address"] + ", " + df.at[i, "city"] + ", Spain"
        coordinates = get_coordinates(address_complete)
        df.at[i, "latitude"]= float(coordinates[1])
        df.at[i, "longitude"]= float(coordinates[0])

In [ ]:
for i in df.index.values.tolist():
    if df.at[i, "latitude"] < 20:
        latitude = df.at[i, "latitude"]
        longitude = df.at[i, "longitude"]
        df.at[i, "latitude"] = longitude
        df.at[i, "longitude"] = latitude

In [ ]:
df['latitude'] = df['latitude'].fillna(0)

In [ ]:
df.drop(df[df.latitude < 30].index, inplace=True)

In [ ]:
df.drop(df[df.longitude < -10].index, inplace=True)

In [ ]:
df.to_csv("Output/spanish_companies.csv")

In [ ]:
df = pd.read_csv('Output/spanish_companies.csv')

In [ ]:
df = df.set_index('Unnamed: 0')
df.index.name = None

In [ ]:
df

In [ ]:
position = ["Designers", "UI/UX Engineers", "Frontend Developers", "Data Engineers", 
             "Backend Developers", "Account Managers", "Maintenance", "Executives", "CEO"]
number_of_employees = [20, 5, 10, 15, 5, 20, 1, 10, 1]
salary = [30000, 33000, 33000, 44000, 39000, 25000, 20000, 60000, 80000]

In [ ]:
employees_dict = {"position":position, "number_of_employees":number_of_employees, "salary":salary}

In [ ]:
df2 = pd.DataFrame.from_dict(employees_dict)

In [ ]:
df2

In [ ]:
requirements = ["Design company", "Schools", "Successful tech startup", "Starbucks", "Airport", "Party", 
                "Vegan restaurant", "Basketball stadium", "Dog hairdresser"]
related_employees = ["Designers", "30% of All", "Frontend Developers/Backend Developers", "Executives", "Account Managers", "All", 
                     "CEO", "Maintenance", "All"]
weight = [80, 100, 80, 40, 100, 10, 300, 500, 1]

In [ ]:
requirements_dict = {"requirements":requirements, "related_employees":related_employees, "weight":weight}

In [ ]:
df3 = pd.DataFrame.from_dict(requirements_dict)

In [ ]:
df3

In [ ]:
final_weight = []
everyone = list(df2["position"].values)
related_employees = [["Designers"], everyone, ['Frontend Developers', 'Backend Developers'], ['Executives'], ['Account Managers'], everyone, ['CEO'], ['Maintenance'], everyone] 
for i, employee_types in enumerate(related_employees):
    weight = df3.at[i, "weight"]
    final = 0
    for employee_type in employee_types:
        number_of_employees = list(df2.loc[df2['position'] == employee_type]["number_of_employees"].values)[0]
        salary = list(df2.loc[df2['position'] == employee_type]["salary"].values)[0]
        final += weight*number_of_employees*salary
    if i == 1: 
        final *= 0.3
    final_weight.append(int(final))

In [ ]:
df3['final_weight'] = final_weight

In [ ]:
df3

In [ ]:
df

In [ ]:
def get_results_from_foursquare (query, location, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={location[0]}%2C{location[1]}&radius=100000&sort=RELEVANCE&limit={limit}"
    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"]

In [ ]:
query = "school"
limit = 10
school_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        starbucks_distance.append(min(distances))
    else:
        starbucks_distance.append(100000)
df['school_distance'] = school_distance

In [ ]:
df.to_csv("Output/spanish_companies_school.csv")

In [ ]:
query = "starbucks"
limit = 10
starbucks_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        starbucks_distance.append(min(distances))
    else:
        starbucks_distance.append(100000)
df['starbucks_distance'] = starbucks_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks.csv")

In [ ]:
query = "airport"
limit = 10
airport_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        airport_distance.append(min(distances))
    else:
        airport_distance.append(100000)
df['airport_distance'] = airport_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport.csv")

In [ ]:
query = "vegan"
limit = 10
vegan_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        vegan_distance.append(min(distances))
    else:
        vegan_distance.append(100000)
df['vegan_distance'] = vegan_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan.csv")

In [ ]:
query = "Basketball stadium"
limit = 10
basketball_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        basketball_distance.append(min(distances))
    else:
        basketball_distance.append(100000)
df['basketball_distance'] = basketball_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball.csv")

In [ ]:
query = "party"
limit = 10
party_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        party_distance.append(min(distances))
    else:
        party_distance.append(100000)
df['party_distance'] = party_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party.csv")

In [ ]:
query = "pet service"
limit = 10
dog_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        dog_distance.append(min(distances))
    else:
        dog_distance.append(100000)
df['dog_distance'] = dog_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog.csv")

In [ ]:
query = "Design"
limit = 10
design_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        design_distance.append(min(distances))
    else:
        design_distance.append(100000)
df['design_distance'] = design_distance

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog_design.csv")

In [ ]:
query = "tech startup"
limit = 10
tech_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        tech_distance.append(min(distances))
    else:
        tech_distance.append(100000)
df['tech_distance'] = tech_distance

In [ ]:
df.to_csv("Output/spanish_companies_all.csv")

In [ ]:
cols = df.columns.tolist()
cols2 = cols[0:6] + [cols[-3]] + [cols[6]] + [cols[-2]] + cols[7:9] + [cols[-5]] + cols[9:11] + [cols[-4]]
df = df[cols2]

In [ ]:
df.to_csv("Output/spanish_companies_ordered.csv")

In [ ]:
sum_of_distances = []
list_of_columns_to_add = list(df.columns)[6:]
for index in df.index.values.tolist():
    sum_ = 0
    i = 0
    for column in list_of_columns_to_add:
        weight = df3.at[i, "final_weight"]
        sum_ += df.at[index, column]*weight
        i += 1
    sum_of_distances.append(sum_)
df['sum_of_distances'] = sum_of_distances

In [ ]:
df.to_csv("Output/spanish_companies_sum_of_distances.csv")

In [ ]:
df = pd.read_csv('Output/spanish_companies_sum_of_distances.csv')
df = df.set_index('Unnamed: 0')
df.index.name = None
df

In [ ]:
idx_min = df["sum_of_distances"].idxmin()

In [ ]:
val_min = df["sum_of_distances"][df['city']=="Valencia"].idxmin()

In [ ]:
bcn_min = df["sum_of_distances"][df['city']=="Barcelona"].idxmin()

In [ ]:
mad_min = df["sum_of_distances"][df['city']=="Madrid"].idxmin()

In [ ]:
venue_location = [df.at[idx_min, "latitude"], df.at[idx_min, "longitude"]]
map_1 = folium.Map(venue_location, zoom_start=17)

In [ ]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_1)

In [ ]:
design_location = [39.469913, -0.373776]
coordinates = [venue_location, design_location]
distance = df.at[idx_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_1.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_1)

In [ ]:
school_location = [39.468421, -0.378141]
coordinates = [venue_location, school_location]
distance = df.at[idx_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_1.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_1)

In [ ]:
tech_location = [39.4698, -0.373265]
coordinates = [venue_location, tech_location]
distance = df.at[idx_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_1.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_1)

In [ ]:
cafe_location = [39.467075, -0.374612]
coordinates = [venue_location, cafe_location]
distance = df.at[idx_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_1.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_1)

In [ ]:
airport_location = [39.491404, -0.473393]
coordinates = [venue_location, airport_location]
distance = df.at[idx_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_1.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_1)

In [ ]:
party_location = [39.463429, -0.372434]
coordinates = [venue_location, party_location]
distance = df.at[idx_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_1.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_1)

In [ ]:
vegan_location = [39.469071, -0.377547]
coordinates = [venue_location, vegan_location]
distance = df.at[idx_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_1.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_1)

In [ ]:
basketball_location = [39.451829, -0.366131]
coordinates = [venue_location, basketball_location]
distance = df.at[idx_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_1.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_1)

In [ ]:
dog_location = [39.459688, -0.383472]
coordinates = [venue_location, dog_location]
distance = df.at[idx_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_1.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_1)

In [ ]:
map_1

In [ ]:
venue_location = [df.at[bcn_min, "latitude"], df.at[bcn_min, "longitude"]]
map_2 = folium.Map(venue_location, zoom_start=17)

In [ ]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_2)

In [ ]:
design_location = [41.389258, 2.128659]
coordinates = [venue_location, design_location]
distance = df.at[bcn_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_2.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_2)

In [ ]:
school_location = [41.39262, 2.129386]
coordinates = [venue_location, school_location]
distance = df.at[bcn_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_2.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_2)

In [ ]:
tech_location = [41.388623, 2.130295]
coordinates = [venue_location, tech_location]
distance = df.at[bcn_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_2.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_2)

In [ ]:
cafe_location = [41.388763, 2.130817]
coordinates = [venue_location, cafe_location]
distance = df.at[bcn_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_2.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_2)

In [ ]:
airport_location = [41.289084, 2.073963]
coordinates = [venue_location, airport_location]
distance = df.at[bcn_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_2.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_2)

In [ ]:
party_location = [41.389888, 2.135753]
coordinates = [venue_location, party_location]
distance = df.at[bcn_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_2.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_2)

In [ ]:
vegan_location = [41.377077, 2.122568]
coordinates = [venue_location, vegan_location]
distance = df.at[bcn_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_2.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_2)

In [ ]:
basketball_location = [41.380163, 2.120141]
coordinates = [venue_location, basketball_location]
distance = df.at[bcn_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_2.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_2)

In [ ]:
dog_location = [41.386995, 2.130976]
coordinates = [venue_location, dog_location]
distance = df.at[bcn_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_2.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_2)

In [ ]:
map_2

In [ ]:
venue_location = [df.at[mad_min, "latitude"], df.at[mad_min, "longitude"]]
map_3 = folium.Map(venue_location, zoom_start=17)

In [ ]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_3)

In [ ]:
design_location = [40.422644, -3.709948]
coordinates = [venue_location, design_location]
distance = df.at[mad_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_3.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_3)

In [ ]:
school_location = [40.424884, -3.714747]
coordinates = [venue_location, school_location]
distance = df.at[mad_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_3.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_3)

In [ ]:
tech_location = [40.423712, -3.707406]
coordinates = [venue_location, tech_location]
distance = df.at[mad_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_3.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_3)

In [ ]:
cafe_location = [40.42332, -3.710883]
coordinates = [venue_location, cafe_location]
distance = df.at[mad_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_3.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_3)

In [ ]:
airport_location = [40.484793, -3.568002]
coordinates = [venue_location, airport_location]
distance = df.at[mad_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_3.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_3)

In [ ]:
party_location = [40.432319, -3.698027]
coordinates = [venue_location, party_location]
distance = df.at[mad_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_3.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_3)

In [ ]:
vegan_location = [40.424878, -3.715591]
coordinates = [venue_location, vegan_location]
distance = df.at[mad_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_3.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_3)

In [ ]:
basketball_location = [40.430009, -3.708137]
coordinates = [venue_location, basketball_location]
distance = df.at[mad_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_3.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_3)

In [ ]:
dog_location = [40.436567, -3.712888]
coordinates = [venue_location, dog_location]
distance = df.at[mad_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_3.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_3)

In [ ]:
map_3